# Q: Can we use levenshtein distance to identify writing samples that merely reflect the language of the prompt?

### Edit Distance

Levenshtein distance computes the edit distance between two strings. Allowable operations are deletions, insertions, and substitutions.

This will need to be prompt specific. So we will run the code separately for each prompt, for each level.

We run into a factorial situation if we compare every sample with every other sample (N!). 

A potential way around this computationally expensive approach would be to create a median string.

Imagine we have some responses that are more or less echos of the prompt. These responses will be similar to each other. We can compute the median string from this bag of highly similar responses. Rather than comparing the next response to each of these, we can compare it to the median string. There will probably be one median string that closely approximates the prompt (in terms of edit distance, in reality it will be a gibberish stream of characters).

Anyway, that's a lot of writing. Let's see what happens when we code it :)

In [49]:
import Levenshtein as lev
from IPython.display import display
import pandas as pd

In [2]:
all_levels = pd.read_csv('all_levels.csv', low_memory=False)
print(len(all_levels))

549281


In [35]:
test_topic = all_levels[all_levels['topic_id']==3535].dropna().reset_index(drop=True)
print(len(test_topic))

41950


For testing the idea, let's just "train" a median string on a training set and "test" on a testing set.

In [44]:
med_str = lev.median(test_topic['text'][0:100])
print(med_str)

print('***By edit distance:')
for text in test_topic['text'][100:200]:
    if lev.distance(med_str, text) < 60:
        print(text)

print('***By ratio:')
# The same thing, but using ratio instead of distance.
for text in test_topic['text'][100:200]:
    if lev.ratio(med_str, text) > .5:
        print(text)

He am teacher    name is  an  i n  am e no m e  in y a n o    in a e is o d nu in e  an 
***By edit distance:
Hi  teacher.  My   name's    Cow  milk,i'm  26  years   old ,  I  come   from  Jiangsu   changzhou
hi teacher:   my name is leo,i am 20 old. i like swiming,dancing .
Hello,teacher,My name's Anly luo,I come from China Chongqing
Dear teacher:      My name's Wumaocheng.      My phone's number is 13372181885
Dear teacher:                      My name's ZhangHui,I'm Chinese.I'm a new student.
***By ratio:
Hi  teacher.  My   name's    Cow  milk,i'm  26  years   old ,  I  come   from  Jiangsu   changzhou
Dear teacher:    My  name is Amy.   I'm  form to Tw,I 'm  live  in NanChang  city.  I'm  fourty  seven year old.
hi teacher:   my name is leo,i am 20 old. i like swiming,dancing .
Hello:       My  name's  Deng Quang Liu. I'm from  to HeFei.  I  want to speeking english . What's your  name ?
Hello , teacher; My name is annis. Nice to meet you. I like English very much. Thanks for you t

### FAIL

This simply does not work (for this prompt). We can identify "Hello teacher", "my name is", and "I'm from", but I don't think we want to clean these samples. (bummer)

Kind of interesting that most characters with high variance are averaged to "e" in the median string. Reminds me of __[Gadsby](https://en.wikipedia.org/wiki/Gadsby_(novel))__

Let's try a more advanced prompt, just in case the results look a bit more interesting.

In [48]:
# display(all_levels[all_levels['lvl']==7])
test_topic = all_levels[all_levels['topic_id']==3124].dropna().reset_index(drop=True)


med_str = lev.median(test_topic['text'][0:100])
print('Median string is:', med_str, end='\n\n')
for text in test_topic['text'][100:200]:
    if lev.ratio(med_str, text) > .55: # I think ratio is less sensitive (than distance) to discrepancies in text length... I should verify this.
        print(text, end='\n\n')

Median string is: Joe ano nale  Lti e ion l oan ge ion a n ues tor  na no ant ring Mon aiter ion  ale ion an  er eting Manager  o a ion a e aon ain  eo r e tine san age ri  e aon dusies in cale ae d esin a d erts mana e projects , or with sales to build client base.  e aine ion  ie  irem nts : e  e ar etin  or similar  e aere ene ant experience  a a re ran e a n  roe tie  an e er ant 

We are looking for a  regional marketing manager. Job duties:Design adverts, manage projects, work with sales to build client base . Minimum requirements: BA marketing or similar,3 yrs relevant experience . Salary range: $35,000 to $40,000. John Tiles LTD is manufacturing idustry and locating  Manchester.

Regional Marketing Manager is needed by John Tiles Ltd., which is located in Manchester. Everyone who is reliable,efficiently,honest and experienced on marketing are needed for this job. Job Duties:Design adverts,Manage projects,Work with sales to build client base Minimum Requirments:BA Marketing or s

### Hot Diggity!

We are identifying phrases that were almost certainly present in the prompt: "John tiles Ltd is a manufactoring company", "looking for a regional marketing manager", "design adverts, manage projects, and work with sales to build a client base", "pay runs from 35,000 to 40,000 depending on experience" etc. etc.

Still. I don't think we want to clean these samples...
If the task was to recreate the prompt based on thousands of responses, this method could probably be improved to do that, for some prompts. That doesn't seem like a realistic use case.


I think it would make more sense to fuzzy match Ngrams, rather than the whole string. We could search for identical Ngrams (say, 5-grams or 6-grams) in all the responses. Once we have a list of somewhat frequent N-grams, we could fuzzy match (which also uses Levenshtein distance) all the responses for those N-grams. I'm going to hold off on this because I don't really see the merit at the data cleaning stage. These methods could be helpful if we build TF-IDF vectors. Basically, these N-grams that come from the prompt are certianly less useful predictors than other strings, but I don't think they should invalidate the entire sample, so we might want to think about a weighting scheme for them. TF-IDF does the weighting for us, and fuzzy matching (if we figure out a reasonable implementation) would handle spelling errors and slight variations on the Ngrams.

The __[PolyFuzz package](https://towardsdatascience.com/string-matching-with-bert-tf-idf-and-more-274bb3a95136)__ seems like the way to go for grouping samples.